In [2]:
!pip install awswrangler --q

Keyring is skipped due to an exception: 'keyring.backends'


In [3]:
import os
import io
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import datetime

import awswrangler as wr
#import dask.dataframe as dd

import warnings
warnings.filterwarnings("ignore")

### local processing 

In [4]:
path = "/root/Lapse/data/"

#### pfmc inf

In [5]:
filenames = glob.glob(path + "*pfmc*.csv")
print(filenames)

['/root/Lapse/data/vul_pfmc_inf_monthly_2022_4.csv', '/root/Lapse/data/vul_pfmc_inf_monthly_2022_2.csv', '/root/Lapse/data/vul_pfmc_inf_monthly_2022_6.csv', '/root/Lapse/data/vul_pfmc_inf_monthly_2022_1.csv', '/root/Lapse/data/vul_pfmc_inf_monthly_2022_5.csv', '/root/Lapse/data/vul_pfmc_inf_monthly_2022_3.csv']


In [6]:
#list of all df
dfs = []
for f in filenames: 
    print(f)
    d = pd.read_csv(f, encoding="ISO-8859-1") 
    dfs.append(d) 
pfmc = pd.concat(dfs, ignore_index=True)
del dfs
print(pfmc.shape)

/root/Lapse/data/vul_pfmc_inf_monthly_2022_4.csv
/root/Lapse/data/vul_pfmc_inf_monthly_2022_2.csv
/root/Lapse/data/vul_pfmc_inf_monthly_2022_6.csv
/root/Lapse/data/vul_pfmc_inf_monthly_2022_1.csv
/root/Lapse/data/vul_pfmc_inf_monthly_2022_5.csv
/root/Lapse/data/vul_pfmc_inf_monthly_2022_3.csv
(4933250, 234)


In [7]:
pfmc['val_dt']= pfmc['val_dt'].astype('datetime64[ns]')

# remove NAT time 
pfmc = pfmc.dropna(subset=['val_dt'])
pfmc['pfmc_cur_month'] = pfmc['val_dt'].map(lambda x: x.strftime('%Y-%m'))

In [8]:
pfmc.groupby('pfmc_cur_month')['agmt_pkge_id'].count()

pfmc_cur_month
2022-01    827272
2022-02    827819
2022-03    823926
2022-04    820633
2022-05    829921
2022-06    803676
Name: agmt_pkge_id, dtype: int64

#### pfmc events

In [9]:
filenames = glob.glob(path + "*events_monthly*.csv")
print(filenames)

['/root/Lapse/data/events_monthly_2022_7.csv', '/root/Lapse/data/events_monthly_2022_5.csv', '/root/Lapse/data/events_monthly_2022_9.csv', '/root/Lapse/data/events_monthly_2022_4.csv', '/root/Lapse/data/events_monthly_2022_8.csv', '/root/Lapse/data/events_monthly_2022_6.csv']


In [10]:
#list of all df
dfs = []
for f in filenames: 
    print(f)
    d = pd.read_csv(f, encoding="ISO-8859-1") 
    dfs.append(d) 
events = pd.concat(dfs, ignore_index=True)
del dfs

/root/Lapse/data/events_monthly_2022_7.csv
/root/Lapse/data/events_monthly_2022_5.csv
/root/Lapse/data/events_monthly_2022_9.csv
/root/Lapse/data/events_monthly_2022_4.csv
/root/Lapse/data/events_monthly_2022_8.csv
/root/Lapse/data/events_monthly_2022_6.csv


In [11]:
#import dateutil.relativedelta #evnt_dt or prcs_dt?
events['prcs_dt']= events['prcs_dt'].astype('datetime64[ns]')
events['evnt_cur_month'] = events['prcs_dt'].map(lambda x: x.strftime('%Y-%m'))
events['evnt_3_month_ago'] =  ((events['prcs_dt'] - pd.DateOffset(months=3)).dt.to_period("M")).astype(str)

In [12]:
# indicator
events['lapse_ind'] = (np.where(events['evnt_desc'].str.contains('Lapse', na=False),1, 0)).astype(int)
events['surrender_ind'] = (np.where(events['evnt_desc'].str.contains('Surrender',na=False),1, 0)).astype(int)
events['reinstate_ind'] = (np.where(events['evnt_desc'].str.contains('True Reinstatement',na=False),1, 0)).astype(int)

### join 

In [13]:
# Merge pfmv and events
df = pfmc.merge(events[['agmt_pkge_id','evnt_desc','lapse_ind', 'surrender_ind', 'reinstate_ind','evnt_3_month_ago', 
                        'evnt_cur_month']], 
              how='left', 
              left_on=['agmt_pkge_id', 'pfmc_cur_month'], 
              right_on=['agmt_pkge_id', 'evnt_3_month_ago'], 
              suffixes=('', '_remove'))

df.drop([i for i in df.columns if 'remove' in i],axis=1, inplace=True)

In [14]:
print(df.shape)

(4933247, 241)


In [15]:
# this causes mixed type
del df["iss_rmo"]
del df['limra_lob_desc']

### write back as parquet

In [16]:
# import pyarrow
# import pyarrow.parquet as pq

# table = pyarrow.Table.from_pandas(df)

In [17]:
%%time
import awswrangler as wr

wr.s3.to_parquet(
    df=df,
    path='s3://cdo-ililapse-364524684987-bucket/x266754/lapse/pfmc_jan_jun.parquet'
    #dataset=True
)

CPU times: user 1min 38s, sys: 23.2 s, total: 2min 2s
Wall time: 1min 44s


{'paths': ['s3://cdo-ililapse-364524684987-bucket/x266754/lapse/pfmc_jan_jun.parquet'],
 'partitions_values': {}}

### Aws Set-up

In [18]:
session = boto3.Session()
s3 = session.client('s3')

NameError: name 'boto3' is not defined

### Test: Read parquet from S3 bucket location

In [ ]:
bucket_name = "cdo-ililapse-364524684987-bucket"
file_path = "x266754/lapse/"
#s3_path = "s3://"+bucket_name+file_path
response = s3.list_objects(Bucket = bucket_name, Prefix = file_path + 'pfmc')

In [ ]:
%%time

df = []
for i in range(len(response['Contents'])):
    key = response['Contents'][i]['Key']
    obj = s3.get_object(Bucket = bucket_name, Key = key)
    df_temp = pd.read_parquet(io.BytesIO(obj['Body'].read()))
    df.append(df_temp)
    print(i,' = ',response['Contents'][i]['Key'])
df = pd.concat(df, ignore_index = True)

In [ ]:
df.head()

In [ ]:
df.shape